<a href="https://www.kaggle.com/code/daiqing2009/effective-and-efficient-intrusion-detection?scriptVersionId=212414383" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://www.kaggle.com/code/daiqing2009/effective-and-efficient-intrusion-detection?scriptVersionId=211743249" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://colab.research.google.com/github/daiqing2009/CP640/blob/main/project/code/EE_intrusion_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of Contents
The goal of this project to measure the effectiveness of common machine learning techniques in differentiating malicious traffic from benign one and identifying corresponding attacks categories and sub-categories. Meanwhile, the project will challenge the requirement to detect the intrusion traffic efficiently as required in real-world situations, especially how models trained on small data perform on larger data.
* Effective Machine Learning
    * Import isproportional Data sampling
    * EDA & Preprocessing
    * Model Selection and Evaluation
* Efficient Machine Learning
    * Model performance on different scale of sampling data
    * Potential integration with IPS(Intrusion Prevention System)


# Effective Machine Learning
In this section, dataaset will be imported. Then Exploratory Data Analysis(EDA) and corresponding preprocessing will conducted before model training and evaluation.

In [ ]:
import sklearn as sk
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, accuracy_score, recall_score, precision_score, f1_score

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

RANDOM_STATE = 42
# for lib in [pd, sk, sns, sv, optuna, tf, keras]:
for lib in [pd, sk, sns]:
    print(f"Using {lib.__name__} {lib.__version__}")

# define the thresthold for proprocessors
VAR_THR = 0.025
CORR_THR = 0.93

## Importing Data
The importing subsection try to be competible in all environments, including Colab, Kaggle and local dev. The original dataset is on https://www.kaggle.com/datasets/daiqing2009/ciciot2023-disprop-sampling

In [ ]:
import os

def detect_environment():
    """Detects the environment: Kaggle, Google Colab, or Local."""
    if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
        return "Kaggle"
    elif "COLAB_GPU" in os.environ:
        return "Google Colab"
    else:
        return "Local"

def create_folder_if_not_exists(path):
    """Creates a folder if it does not already exist."""
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Folder created at: {path}")
    else:
        print(f"Folder already exists at: {path}")

def setup_kaggle_credentials():
    """Ensures Kaggle API credentials are available."""
    if detect_environment() == "Google Colab":
      from google.colab import userdata
      os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
      os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
    elif detect_environment() == "Local":
      kaggle_json_path = os.path.expanduser("~/.kaggle/kaggle.json")
      if not os.path.exists(kaggle_json_path):
        # Prompt user to upload `kaggle.json`
        print("Please upload your `kaggle.json` file.")
        from google.colab import files
        uploaded = files.upload()
        with open(kaggle_json_path, "wb") as f:
            f.write(uploaded['kaggle.json'])
      os.chmod(kaggle_json_path, 0o600)  # Set correct permissions for the file

def download_kaggle_resource(input_path, resource_name):
    """
    Downloads a dataset or notebook output from Kaggle using the resource name.
    Automatically unzips if needed and logs useful diagnostic information.
    """
    from kaggle.api.kaggle_api_extended import KaggleApi
    import os

    setup_kaggle_credentials()

    # Initialize Kaggle API
    api = KaggleApi()
    api.authenticate()

    try:
        # Attempt to download as a dataset
        print(f"Attempting to download dataset: {resource_name}")
        api.dataset_download_files(resource_name, path=input_path, unzip=True)
        print(f"Dataset downloaded and extracted to {input_path}")
    except Exception as dataset_error:
        print(f"Dataset not found or inaccessible: {dataset_error}. Attempting as notebook output.")
        try:
            # Attempt to download as notebook output
            print(f"Attempting to download notebook output: {resource_name}")
            api.kernel_output(resource_name, path=input_path, unzip=True)
            print(f"Notebook output downloaded and extracted to {input_path}")
        except Exception as notebook_error:
            print(f"Failed to download resource: {resource_name}.")
            print(f"Dataset error: {dataset_error}")
            print(f"Notebook output error: {notebook_error}")
            print("Please verify the resource name, access permissions, and your Kaggle API credentials.")

# Step 1: Detect environment
environment = detect_environment()
print(f"Environment detected: {environment}")

# Step 2: Create input folder
if environment == "Kaggle":
    input_path = "/kaggle/input"
elif environment == "Google Colab":
    input_path = "/content/input"
elif environment == "Local":
    input_path = "./input"
else:
    raise ValueError("Unknown environment detected!")

create_folder_if_not_exists(input_path)
print(f"Input folder path: {input_path}")

# Step 3: Create output folder
if environment == "Kaggle":
    output_path = "/kaggle/working/output"
elif environment == "Google Colab":
    output_path = "/content/output"
elif environment == "Local":
    output_path = "./output"
else:
    raise ValueError("Unknown environment detected!")

create_folder_if_not_exists(output_path)
print(f"Output folder path: {output_path}")

resource_name = "daiqing2009/ciciot2023-disprop-sampling"
# Step 4: Download dataset or notebook output
if environment != "Kaggle":
    pass
    # resource_name = input("Enter the Kaggle dataset name or notebook name to download: ")
    download_kaggle_resource(input_path, resource_name)

# initialize the data prefix
if environment == "Kaggle":
    dataprefix = input_path + "/" + resource_name.split("/")[1]
elif environment == "Google Colab":
    dataprefix = input_path
elif environment == "Local":
    dataprefix = input_path
# mount cloud drive if necessary
if environment == "Google Colab":
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
# 0.1 percent as size xs
df2_xs = pd.read_csv(f"{dataprefix}/0.001_dist_percent_2classes.csv")
df8_xs = pd.read_csv(f"{dataprefix}/0.001_dist_percent_8classes.csv")
df34_xs = pd.read_csv(f"{dataprefix}/0.001_dist_percent_34classes.csv")

# 0.5 percent as size s
df2_s = pd.read_csv(f"{dataprefix}/0.005_dist_percent_2classes.csv")
df8_s = pd.read_csv(f"{dataprefix}/0.005_dist_percent_8classes.csv")
df34_s = pd.read_csv(f"{dataprefix}/0.005_dist_percent_34classes.csv")

# 1 percent as size m
# df2_m = pd.read_csv("/kaggle/input/disproportionate-sampling-dataset-for-ciciot2023/0.01_dist_percent_2classes.csv")
df8_m = pd.read_csv(f"{dataprefix}/0.01_dist_percent_8classes.csv")
# df34_m = pd.read_csv("/kaggle/input/disproportionate-sampling-dataset-for-ciciot2023/0.01_dist_percent_34classes.csv")

In [ ]:
# setting the default dataset
df2 = df2_xs
df8 = df8_xs
df34 = df34_xs

In [ ]:
# extract the label value
label2=df2["benign"]
df2 = df2.drop("benign", axis=1)
label8=df8["label"]
df8 = df8.drop("label", axis=1)
label34=df34["label"]
df34 = df34.drop("label", axis=1)

## EDA & Preprocessing
**Summary Table of EDA Methods and Preprocessing**
Following table is a brief summary of common Exploratory Data Analysis (EDA) and corresponding Preprocessing methods.

| EDA Method|Typical Preprocessing/Feature Engineering|Relevancy to current probelm|
|--|--|--|
|Data Cleaning|Handle missing values, duplicates, and outliers.|outlier is the challenge of the dataset, capping techniques are applied to |
|Univariate Analysis|Normalize/encode features, transform skewed data.|show skewness of the data and visualize with histogram, boxplot, drop column with limited info|
|Multivariate Analysis|Normalize, reduce multicollinearity, apply dimensionality reduction.|show Correlation table and reduce dimension with PCA/t-CNS|
|Target Variable Analysis|Balance classes, transform skewed distributions.|the label class is highly biased , espcially for (D)DOS. Dispropotional Sampling is used to balance class |
|Time Series Analysis|Interpolate missing data, extract date/time features, apply smoothing.|N/A, since the dataset is already engineered and temporal information has been compacted|
The preprocessing will be conducted in following steps:


1. OneHotEncoding for categorical features.
2. Drop features of both low variance and low correlation with target label.
3. Drop highly correlated features
4. Reduce dimension Using PCA

Finally, all these steps are pipelined using scikit learn functions.



In [ ]:
# Calculate percentages for each unique value
value_counts = df2['protocol_type'].value_counts(normalize=True) * 100

# Sort percentages from highest to lowest
sorted_percentages = value_counts.sort_values(ascending=False)

# Get top 10 percentages (or fewer if there are less than unique values)
top_percentages = sorted_percentages.head(10)

# Plot histogram for the top values sorted by percentage
plt.bar(top_percentages.index, top_percentages.values)
plt.title("Top Percentages Distribution (Highest to Lowest)")
plt.xlabel("protocol_type")
plt.ylabel("Percentage (%)")
plt.xticks(top_percentages.index)  # Ensure x-axis labels show only top values
plt.show()

# Print the top sorted percentages
print("Top 20 Percentages (from Highest to Lowest):")
print(sorted_percentages[:20])

In [ ]:
# Step 1: OneHotEncoding for categorical features
categorical_features = ['protocol_type']
categorical_transformer = OneHotEncoder(sparse_output=False)
encoded_categorical = categorical_transformer.fit_transform(df2[categorical_features])
# print(encoded_categorical)

# Create a DataFrame for the encoded categorical features
encoded_df = pd.DataFrame(encoded_categorical, columns=categorical_transformer.get_feature_names_out(categorical_features))

# Drop the original categorical columns and concatenate the encoded features
df2_filtered = df2.drop(categorical_features, axis=1)
df2_filtered = pd.concat([df2_filtered, encoded_df], axis=1)
df2_filtered.head()

In [ ]:
onehot_cols = ['http','https','dns','telnet','smtp','ssh','irc','tcp','udp','dhcp','arp','icmp','ipv','llc']
onehot_cols.extend(encoded_df.columns.tolist())
# print(f'onehot_cols:{onehot_cols}')
from scipy.stats import chi2_contingency

def cramers_v(contingency_table):
    """Calculate Cramér's V for a contingency table."""
    chi2, _, _, _ = chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    return np.sqrt(chi2 / (n * (min(contingency_table.shape) - 1)))

def filter_strong_association_features(df, target, threshold=0.3):
    """
    Filter binary/categorical features strongly associated with a categorical target label.

    Parameters:
        df (pd.DataFrame): DataFrame containing features and the target label.
        target_col (str): Name of the categorical target label column.
        threshold (float): Minimum Cramér's V value to consider a strong association.

    Returns:
        list: List of feature names with strong association.
    """
    strong_features = []

    # Loop through features in the DataFrame
    for col in df.columns:
        # if col == target_col:  # Skip the target column
        #     continue
        if df[col].nunique() <= 10:  # Check if the column is categorical/binary
            # Create a contingency table
            contingency_table = pd.crosstab(df[col], target)

            # Calculate Cramér's V
            cramer_v_value = cramers_v(contingency_table)

            # If Cramér's V exceeds the threshold, consider it strongly associated
            if cramer_v_value >= threshold:
                strong_features.append(col)

    return strong_features

cols_valueable = filter_strong_association_features(df2_filtered,label34, threshold=0.3)
print(f'cols_valueable in onehotcode:{cols_valueable}')

In [ ]:
# filter out columns with uniform distribution(always the same value)
skewness = df2_filtered.skew()
# print(skewness)

df2_ls = df2_filtered.loc[:,skewness<0.00001]

# Create a grid of subplots
fig, axes = plt.subplots(3, 3, figsize=(9, 9))  # 3 rows, 3 columns
axes = axes.flatten()  # Flatten the axes array for easy indexing

# Plot each column's histogram in the corresponding subplot
for i, column in enumerate(df2_ls.columns):
    # Count the occurrences of True and False
    value_counts = df2_ls[column].value_counts()
    # Plot the histogram
    value_counts.plot(kind='bar', color=['skyblue', 'orange'], ax=axes[i])
    axes[i].set_title(f'{column}')
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Count')
    axes[i].set_xticks([0, 1])
    axes[i].set_xticklabels(['False', 'True'], rotation=0)

# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
# analyze the colunmn with high abs skew values
df2_hs = df2_filtered.loc[:,abs(skewness)>50]

# print(df2_hs.head(10))

# # Identify continuous columns (e.g., numeric columns)
continuous_columns = df2_hs.select_dtypes(include=['float64', 'int64']).columns

# Set up the plotting
plt.figure(figsize=(10, 10))

# delete columns with too high skewness
from scipy.stats import mstats

df_winsorized = df2_hs[continuous_columns].apply(lambda x: mstats.winsorize(x, limits=[0.05, 0.05]))

# Loop through continuous columns and plot histograms
for i, col in enumerate(continuous_columns, 1):
    plt.subplot(5,5, i)
    sns.histplot(df_winsorized[col], kde=True, bins=10, color='skyblue', edgecolor='black')  # histogram with KDE
    plt.title(f'{col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Step 2: drop features of both low variance and low correlation with target label.
var_thr = VarianceThreshold(threshold = VAR_THR) #Removing both constant and quasi-constant
var_thr.fit(df2_filtered)

var_thr.get_support()
concol = [column for column in df2_filtered.columns
          if column not in df2_filtered.columns[var_thr.get_support()]]
print(f'columns of low variance: {concol}')

In [ ]:
# remove the feature of both low variance and low
cols_to_drop = [col for col in concol if col not in cols_valueable]
print(f'cols_to_drop: {cols_to_drop}')
df2_normal = df2_filtered.drop(concol,axis =1)
df2_normal.head()

In [ ]:
# show boxplot of parameter with comparatively normal distribution
continuous_columns = df2_normal.select_dtypes(include=['float64', 'int64']).columns

# Create a grid for the boxplots
plt.figure(figsize=(12, 12))

# Loop through each column and plot the boxplot
for i, col in enumerate(continuous_columns, 1):
    plt.subplot(6,6, i)
    sns.boxplot(x=df2_normal[col])
    plt.title(f'{col}')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Correlation Analysis
def show_corr(df):
    corr_matrix = df.corr()

    # Visualize the correlation matrix

    plt.figure(figsize=(20, 20))

    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.xticks(rotation=90)
    plt.title('Correlation Matrix')
    plt.show()
show_corr(df2_normal)

In [ ]:
# Step 3: Drop highly correlated features
from sklearn.base import BaseEstimator, TransformerMixin

class CorrelationFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.9):
        self.threshold = threshold
        self.to_drop_ = None

    def fit(self, X, y=None):
        # Ensure X is a DataFrame
        X = pd.DataFrame(X)

        # Compute the correlation matrix
        corr_matrix = X.corr().abs()

        # Identify highly correlated features
        upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        self.to_drop_ = [column for column in upper_triangle.columns if any(upper_triangle[column] > self.threshold)]
        print(f'column of high correlation({self.threshold}) to be dropped: {self.to_drop_}')
        return self

    def transform(self, X):
        # Ensure X is a DataFrame
        X = pd.DataFrame(X)

        # Drop identified columns
        return X.drop(columns=self.to_drop_, errors='ignore')

    def fit_transform(self, X, y=None):
        self.fit(X, y)
        return self.transform(X)
df_reduced = CorrelationFilter(threshold=CORR_THR).fit_transform(df2_normal)
# df_reduced = CorrelationFilter(threshold=0.9).fit_transform(df2_normal)

In [ ]:
# Step4:reduce dimension Using PCA
from sklearn.decomposition import PCA

def CumuPlot(df):
    # Apply PCA with the adjusted number of components
    pca = PCA()

    df_pca = pca.fit_transform(df)

    # Calculate cumulative variance
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

    # Plot cumulative explained variance
    plt.figure(figsize=(10, 8))
    plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='-', color='b')
    plt.axhline(y=0.90, color='m', linestyle='-.', label='90% Threshold')  # Optional threshold line
    plt.axhline(y=0.95, color='r', linestyle='--', label='95% Threshold')  # Optional threshold line
    plt.xticks(range(1, len(cumulative_variance) + 1))  # Ensure x-ticks align with components
    plt.xlabel('Number of Principal Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title('Cumulative Explained Variance by PCA Components')
    plt.legend()
    plt.grid()
    plt.show()

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_reduced)
CumuPlot(df_scaled)

In [ ]:
# Apply PCA and retain the top n components
n_components = 20 #take 95% variance as target
pca = PCA(n_components=n_components)

df_scaled = pd.DataFrame(df_scaled)
principal_components = pca.fit_transform(df_scaled)

# print(scaler.fit_transform(principal_components)[:5])

# Get PCA components (loadings)
components = pca.components_

df = df_reduced
# Compute the importance of each feature
importance = components.T  # Transpose to make features the rows
importance_scores = pd.DataFrame(importance, columns=[f'PC{i+1}' for i in range(components.shape[0])], index=df.columns)

# Calculate the total contribution of each feature across all PCs
top_n = 20  # Change this to the desired number of features
importance_scores['Total'] = (importance_scores.abs() * pca.explained_variance_ratio_).sum(axis=1)
print(f"Top({top_n}) features of  importance: ")
print(importance_scores['Total'].sort_values(ascending=False).head(top_n))

In [ ]:
# build up pipeline for structured preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

# determine categorical and numerical features
numerical_feature = df2.select_dtypes(include=['int64', 'float64']).columns
# categorical_ix = df2.select_dtypes(include=['object', 'bool']).columns

# Preprocessing for numeric and categorical features
label_cols = ["benign","label"]
skip_cols = categorical_features + label_cols
pass_cols = [col for col in df2.columns if col not in skip_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
    ],remainder='passthrough')

# Create the pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("variance_filter", VarianceThreshold(threshold = VAR_THR)),
    ("correlation_filter", CorrelationFilter(threshold=CORR_THR)),
    ("scaler", StandardScaler()),
    ("pca", pca)
])

## Models Selection & Evaluation
|Model|Type|Advantage|
|--|--|--|
|Logistic Regression|Supervised  |Easy to implement, computational efficient|
|KNN               |Supervised  |widely-used for classification of known number(k) of clusters, can upgrade to outlier robust variant: DBSCAN|
|Random Forest     |Supervised|Ensemble decision trees, robust to skewness of data|
|Auto-encoder      |Unsupervised|Deep Learning model for abnormal detection, which only differentiate benign and attack flow|

Supervised models will be trained and evaluated on 0.1 percent data in classifying 2/8/34 traffic category, including Benign and Attacking case.

In [ ]:
def split(df,label = "label"):

    # Sorting our dataset into features and target
    X = df.drop(label, axis = 1)
    y = df[label]

    # splitting out dataset to train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=RANDOM_STATE)

    # scaling our features
    scaler = StandardScaler()
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_test = scaler.fit_transform(X_test)

    # return X_train, X_test, y_train, y_test
    return scaled_X_train, scaled_X_test, y_train, y_test

In [ ]:
# creating dataframes to store result metrics
columns = ["Logistic Regression", "KNN", "Random Forest"]
index = ["Accuracy", "Recall", "Precision", "F1-Score"]

metrics_2 = pd.DataFrame(index=index, columns=columns)
metrics_8 = pd.DataFrame(index=index, columns=columns)
metrics_34 = pd.DataFrame(index=index, columns=columns)

In [ ]:
# inserts the metrics of the model into the metrics dataframe
def insert_metrics(classes, model_name, metrics):
    if classes == 2:
        metrics_2.loc['Accuracy':'F1-Score', model_name] = metrics
    elif classes == 8:
        metrics_8.loc['Accuracy':'F1-Score', model_name] = metrics
    else:
        metrics_34.loc['Accuracy':'F1-Score', model_name] = metrics

In [ ]:
# displays the Classification Report and Confusion Matrix
# inserts the metrics of the model into the metrics dataframe
def evaluate(model, X_test, y_test, target_names, classes, model_name):
    pred = model.predict(X_test)

    # print(pred[:5])
    # print(f'y_test.type:{type(y_test)}.shape:{y_test.shape} & y_test.type:{type(pred)}.shape:{pred.shape}')
    accuracy = accuracy_score(y_test, pred)
    recall = recall_score(y_test, pred, average='macro')
    precision = precision_score(y_test, pred, average='macro')
    f1 = f1_score(y_test, pred, average='macro')
    insert_metrics(classes, model_name, [accuracy, recall, precision, f1])

    print(classification_report(y_test, pred, digits = 3))

    if classes == 2 or classes == 8:
        plt.figure(figsize = (4, 2), dpi = 300)
        ConfusionMatrixDisplay(np.round(confusion_matrix(y_test, pred, normalize="true"), 2),
                               display_labels = target_names).plot()
        plt.title(model_name)
        plt.xticks(rotation=30)

In [ ]:
# save and load models for testing on different percentage of data
!mkdir models
def pickle_save(model, name):
    model_filename = 'models/' + name + '.pkl'
    with open(model_filename, 'wb') as model_file:
        pickle.dump(model, model_file)

    print(f"Model saved to {name}")

def pickle_load(name):
    model_filename = 'models/' + name + '.pkl'
    try:
        with open(model_filename, 'rb') as model_file:
            model = pickle.load(model_file)
        print(f"Model loaded from {model_filename}")
        return model
    except FileNotFoundError:
        print(f"Error: The file '{model_filename}' does not exist.")
        return None
    except pickle.UnpicklingError:
        print(f"Error: Failed to load the model from '{model_filename}'. The file might be corrupted.")
        return None

In [ ]:
# print(df2_xs)
df_processed = pd.DataFrame(pipeline.fit_transform(df2))
# df_processed = pd.DataFrame(pca.fit_transform(df_reduced))
# df_processed = pd.read_csv("/kaggle/input/pca-transformed/pca_transformed_data_v11.csv")
# print(df_pca.head())
print(df_processed.shape)
df_processed["label"] = label2
scaled_X_train, scaled_X_test, y_train, y_test = split(df_processed)
target_names = ["Attack", "Benign"]
# print(label2.head())

In [ ]:
logreg_model = LogisticRegression(C = 1, max_iter=500, solver='lbfgs', penalty='l2', random_state=42)
logreg_model.fit(scaled_X_train, y_train)

evaluate(logreg_model, scaled_X_test, y_test, target_names, 2, "Logistic Regression")
pickle_save(logreg_model, "logreg_2")

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 8)
knn.fit(scaled_X_train, y_train)

evaluate(knn, scaled_X_test, y_test, target_names, 2, "KNN")
pickle_save(knn, "knn_2")

In [ ]:
# Weight-balanced
# rfc = RandomForestClassifier(n_estimators=100, class_weight="balanced")
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(scaled_X_train, y_train)

evaluate(rfc, scaled_X_test, y_test, target_names, 2, "Random Forest")
pickle_save(rfc, "rfc_2")

In [ ]:
# exporting and displaying the class 2 metrics
metrics_2.to_csv(f'{output_path}/metrics_2.csv', index=True)
metrics_2

### 8 Classes

In [ ]:
df_processed = pd.DataFrame(pipeline.fit_transform(df8))
df_processed['label']=label8
target_names = ["Benign", "BruteForce", "DDoS", "Dos", "Mirai", "Recon", "Spoofing", "Web"]
scaled_X_train, scaled_X_test, y_train, y_test = split(df_processed)

In [ ]:
logreg_model = LogisticRegression(C = 1, max_iter=500, solver='lbfgs', penalty='l2', random_state=42)
logreg_model.fit(scaled_X_train, y_train)

evaluate(logreg_model, scaled_X_test, y_test, target_names, 8, "Logistic Regression")
pickle_save(logreg_model, "logreg_8")

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 8)
knn.fit(scaled_X_train, y_train)

evaluate(knn, scaled_X_test, y_test, target_names, 8, "KNN")
pickle_save(knn, "knn_8")

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, class_weight="balanced")
rfc.fit(scaled_X_train, y_train)

evaluate(rfc, scaled_X_test, y_test, target_names, 8, "Random Forest")
pickle_save(rfc, "rfc_8")

In [ ]:
# exporting and displaying the class 8 metrics
metrics_8.to_csv(f'{output_path}/metrics_8.csv', index=True)
metrics_8

### 34 Classes

In [ ]:
df_processed = pd.DataFrame(pipeline.fit_transform(df34))
df_processed['label']=label34
scaled_X_train, scaled_X_test, y_train, y_test = split(df_processed)
print(y_test.value_counts())
target_names = list(y_test.unique())

In [ ]:
logreg_model = LogisticRegression(C = 1, max_iter=500, solver='lbfgs', penalty='l2', random_state=42)
logreg_model.fit(scaled_X_train, y_train)

evaluate(logreg_model, scaled_X_test, y_test, target_names, 34, "Logistic Regression")
pickle_save(logreg_model, "logreg_34")

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 34)
knn.fit(scaled_X_train, y_train)

evaluate(knn, scaled_X_test, y_test, target_names, 34, "KNN")
pickle_save(knn, "knn_34")

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, class_weight="balanced")
rfc.fit(scaled_X_train, y_train)

evaluate(rfc, scaled_X_test, y_test, target_names, 34, "Random Forest")
pickle_save(rfc, "rfc_34")

In [ ]:
# exporting and displaying the class 34 metrics
metrics_34.to_csv(f'{output_path}/metrics_34.csv', index=True)
metrics_34

In [ ]:
# List of model names
model_names = ["Logistic Regression", "KNN", "Random Forest"]

# List of dataframes
dataframes = [metrics_2, metrics_8, metrics_34]

# List of classes
classes = [2, 8, 34]

# List of metric names
metrics = ["Accuracy", "Recall", "Precision", "F1-Score"]

# Create a color mapping dictionary for each model
model_colors = {
    "Logistic Regression": 'tab:blue',
    "KNN": 'tab:red',
    "Random Forest": 'tab:olive',
}

# Creating subplots for each metric
for metric in metrics:
    fig, axs = plt.subplots(1, len(classes), figsize=(12, 4), sharey=True)
    fig.suptitle(f"{metric} Scores")

    for i, df in enumerate(dataframes):
        axs[i].set_title(f"Class {classes[i]}")

        # Plotting the respective metric score for each model with specified color
        for model_name in model_names:
            color = model_colors[model_name]
            axs[i].bar(model_name, df.loc[metric, model_name], color=color)

        # Set x-axis ticks and labels, rotating labels by 90 degrees
        axs[i].set_xticks(range(len(model_names)))
        axs[i].set_xticklabels(model_names, rotation=30)

    # Save the figure
    plt.savefig(f'{output_path}/{metric}_scores.png', bbox_inches='tight')

    plt.show()

# Efficient Intrution Detection
In real-world application, it requires that the detection will be effective as well as efficient.
One scenario will be analyzed: how models trained in smaller percentage of data perform on larger percentage of data. Though some degree of performance downgrade is observed, the models can stil function and some interesitng findings here:


*   Random Forest perform better for high-density data and worse on low-density ones. The reason behind is likely its tendency of overfitting despite of ensembling the average output of decision tree.
*   KNN perform just the opposite due to its nature to aggregate the local neighbours.




## Evaluation of Models Trained on Smaller Data Subsets Applied to Larger Datasets

In [ ]:
# test model on larger percentage
label8_s = df8_s["label"]
df8 = df8_s.drop(["label"],axis=1)

# Convert NumPy array to DataFrame
df_processed = pd.DataFrame(pipeline.fit_transform(df8))
df_processed['label'] = label8_s
scaled_X_train, scaled_X_test, y_train, y_test = split(df_processed)
target_names = ["Benign", "BruteForce", "DDoS", "Dos", "Mirai", "Recon", "Spoofing", "Web"]

rfc8_load = pickle_load("rfc_8")
evaluate(rfc8_load, scaled_X_test , y_test, target_names, 8, "Random Forest")

knn8_load = pickle_load("knn_8")
evaluate(knn8_load, scaled_X_test , y_test, target_names, 8, "KNN")

In [ ]:
# extract the label value
label8_m = df8_m["label"]
df8 = df8_m.drop(["label"],axis=1)

# Convert NumPy array to DataFrame
df_processed = pd.DataFrame(pipeline.fit_transform(df8))
df_processed['label'] = label8_m
scaled_X_train, scaled_X_test, y_train, y_test = split(df_processed)
target_names = ["Benign", "BruteForce", "DDoS", "Dos", "Mirai", "Recon", "Spoofing", "Web"]

rfc8_load = pickle_load("rfc_8")
evaluate(rfc8_load, scaled_X_test , y_test, target_names, 8, "Random Forest")

knn8_load = pickle_load("knn_8")
evaluate(knn8_load, scaled_X_test , y_test, target_names, 8, "KNN")

## Autoencoder
If time permitted, autoencoder will be used to compare the effect of differentiating benign/malicious traffic with classical models.
Inspired by https://www.kaggle.com/code/alexandrelemercier/quantized-autoencoder-qae-ids-for-iot-devices

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
class AutoencoderAnomalyDetector(nn.Module):
    def __init__(self, input_dim, threshold=0.1):
        super(AutoencoderAnomalyDetector, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )
        self.threshold = threshold

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

    def predict(self, x):
        with torch.no_grad():
            reconstructed = self.forward(x)
            reconstruction_error = torch.mean((x - reconstructed) ** 2, dim=1)
            return (reconstruction_error > self.threshold).int()

    @staticmethod
    def get_device():
        return "cpu"
        # return torch.device('mps' if torch.backends.mps.is_available() else
        #                     'cuda' if torch.cuda.is_available() else 'cpu')

    def train_model(self, dataloader, epochs=20, lr=1e-3):
        device = self.get_device()
        self.to(device)
        optimizer = optim.Adam(self.parameters(), lr=lr)
        criterion = nn.MSELoss()
        self.train()

        epoch_losses = []
        reconstruction_error = []
        for epoch in range(epochs):
            total_loss = 0
            for batch in dataloader:
                data = batch[0].to(device)
                optimizer.zero_grad()
                reconstructed = self.forward(data)
                loss = criterion(reconstructed, data)
                loss.backward()
                optimizer.step()
                reconstruction_error.extend(torch.mean((data - reconstructed) ** 2, dim=1).cpu().detach().numpy())
                total_loss += loss.item()
            avg_loss = total_loss / len(dataloader)
            epoch_losses.append(avg_loss)
            # print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")
        
        self.threshold = np.mean(reconstruction_error) + 1.5 * np.std(reconstruction_error)
        self.plot_learning_curve(epoch_losses)
        self.plot_reconstruction_error_boxplot(reconstruction_error, self.threshold, "Reconstruction Error Distribution")
        return epoch_losses

    def evaluate(self, dataloader, y_true):
        device = self.get_device()
        self.to(device)
        self.eval()
        y_pred = []
        with torch.no_grad():
            for batch in dataloader:
                data = batch[0].to(device)
                preds = self.predict(data)
                y_pred.extend(preds.cpu().numpy())
        # print(f'y_pred:{len(y_pred)} & y_true:{len(y_true)}')
        print(classification_report(y_true, y_pred, digits = 3))
        plt.figure(figsize = (4, 2), dpi = 300)
        target_names = ["Attack", "Benign"]
        ConfusionMatrixDisplay(np.round(confusion_matrix(y_true, y_pred, normalize="true"), 4),
                               display_labels = target_names).plot()
        plt.title(model_name)
        plt.xticks(rotation=30)

    @staticmethod
    def plot_learning_curve(losses):
        plt.figure(figsize=(8, 6))
        plt.plot(range(1, len(losses) + 1), losses, marker='o', linestyle='-', color='b')
        plt.title("Learning Curve")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.grid(True)
        plt.show()
    
    # Plotting function remains unchanged
    @staticmethod
    def plot_reconstruction_error_boxplot(reconstruction_error, threshold, title):
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=reconstruction_error, color='blue')
        plt.axvline(threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {threshold}')
        plt.title(title)
        plt.xlabel('Reconstruction Error')
        plt.legend()
        plt.xlim([0, np.percentile(reconstruction_error, 99)])
        plt.show()

from torch.utils.data import DataLoader, TensorDataset

normal = df2[label2] # Normal data
print(f'normal.shape:{normal.shape}')

# Combine the normal and anomaly data
df_tr = pipeline.fit_transform(normal) # Normal data

# print(f'df_tr.shape:{df_tr.shape}')
# Convert to PyTorch tensors
X = torch.tensor(df_tr, dtype=torch.float32)
# y = torch.tensor(y_train.values, dtype=torch.float32)

# DataLoader
dataset = TensorDataset(X)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Train the model
detector = AutoencoderAnomalyDetector(input_dim=X.shape[1])
train_losses = detector.train_model(dataloader, epochs=40, lr=1e-3)

# Evaluate the model
df_val = pipeline.fit_transform(df2) # Normal data

# Convert to PyTorch tensors
X_val = torch.tensor(df_val, dtype=torch.float32)
y_val = torch.tensor(label2.values, dtype=torch.float32)

# DataLoader
dataset = TensorDataset(X_val)
test_loader = DataLoader(dataset, batch_size=128, shuffle=True)
detector.evaluate(test_loader, y_val)


## Potential integration with IPS(Intrusion Prevention System)
In Kaggle community, there are existing feature importance analysis based on different model. Meanwhile, there exists many fine-tuning framework in the market, such as Optuna, which can find the best parameter for identifying attack traffic in accuracy. Combined with these efforts, it is promising an automatic threshold setting   on configurable parameter of IPS, such as Suricata, will be able to adaptively block the intrusion attempt.

*This can be good research topic for future.*